In [1]:
import torch
import torch.nn as nn
from Exercise1_1 import LQRSolver

class DGMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(DGMNet, self).__init__()
        self.input_dim = input_dim
        self.hidden1 = nn.Linear(input_dim, hidden_dim)
        self.hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, 1)
        self.activation = nn.Tanh()
        
    def forward(self, t, x):
        inputs = torch.cat([t, x], dim=1)  # Concatenate t and x
        x = self.activation(self.hidden1(inputs))
        x = self.activation(self.hidden2(x))
        return self.output(x)



In [2]:
def compute_loss(model, t, x):
    # Use autograd for derivatives
    u = model(t, x)
    u_t = torch.autograd.grad(u.sum(), t, create_graph=True)[0]
    u_x = torch.autograd.grad(u.sum(), x, create_graph=True)[0]

    # Compute second-order derivatives w.r.t. each spatial variable
    u_xx = torch.zeros_like(u)
    for i in range(x.size(1)):
        u_xx += torch.autograd.grad(u_x[:, i:i+1].sum(), x, create_graph=True)[0][:, i:i+1]

    # Example PDE residual: ∂u/∂t + Δu - cu = 0
    c = torch.tensor(1.0)
    residual = u_t + u_xx - c * u
    return torch.mean(residual ** 2)


In [3]:
def sample_points(num_samples, T, L):
    t = (torch.rand(num_samples, 1, dtype = torch.double) * T).requires_grad_(True)
    x = torch.cat([torch.rand(num_samples, 1, dtype = torch.double) * l for l in L], dim=1).requires_grad_(True)
    return t, x



In [4]:
T = 1.0  # Time interval upper bound
L = [2, 3]  # Spatial domain bounds for each dimension

model = DGMNet(input_dim=1+len(L), hidden_dim=100).double()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(501):
    t_samples, x_samples = sample_points(1000, T, L)
    loss = compute_loss(model, t_samples, x_samples)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')



Epoch 0, Loss: 0.04430004020476433
Epoch 50, Loss: 0.00022999352788597571
Epoch 100, Loss: 5.8655271420391025e-05
Epoch 150, Loss: 2.542910066191795e-05
Epoch 200, Loss: 1.849015776882308e-05
Epoch 250, Loss: 1.395339381673666e-05
Epoch 300, Loss: 1.199625209809192e-05
Epoch 350, Loss: 1.181681287640796e-05
Epoch 400, Loss: 1.0073801799686382e-05
Epoch 450, Loss: 8.120149531238523e-06
Epoch 500, Loss: 7.253695998621053e-06


In [65]:
model.eval()

DGMNet(
  (hidden1): Linear(in_features=3, out_features=100, bias=True)
  (hidden2): Linear(in_features=100, out_features=100, bias=True)
  (output): Linear(in_features=100, out_features=1, bias=True)
  (activation): Tanh()
)

In [7]:
# Define matrices for LQR problem
H = torch.tensor([[1.2, 0.8], [-0.6, 0.9]], dtype=torch.double)
M = torch.tensor([[0.5,0.7], [0.3,1.0]], dtype=torch.double)
sigma = torch.tensor([[[0.8],[1.1]]], dtype=torch.double)
C = torch.tensor([[1.6, 0.0], [0.0, 1.1]], dtype=torch.double)
D = torch.tensor([[0.5, 0.0], [0.0, 0.7]], dtype=torch.double)
R = torch.tensor([[0.9, 0.0], [0.0, 1.0]], dtype=torch.double)
T = torch.tensor(1.0, dtype=torch.double)

solver = LQRSolver(H, M, sigma, C, D, R, T=T, method="euler")

In [8]:
# Initialization

t_ends = [0.1,0.9]
t_num = 3
x_ends = [[0.1,1.8],[0.1,2.7]]
x_num = [100,100] # alleviate the calculation workload.

# load_interval_setting = torch.load('Exercise1_2/value_numerical/?x?/'+'interval_setting.pt')
# t_ends = load_interval_setting['t_ends']
# t_num = load_interval_setting['t_num']
# x_ends = load_interval_setting['x_ends']
# x_num = load_interval_setting['x_num']


# Establish meshgrid structure from setting.

t_batch_i = torch.linspace(t_ends[0],t_ends[1],t_num,dtype=torch.double)
t_batch = t_batch_i.repeat_interleave(x_num[0]*x_num[1])

x1 = torch.linspace(x_ends[0][0],x_ends[0][1],x_num[0],dtype=torch.double)
x2 = torch.linspace(x_ends[1][0],x_ends[1][1],x_num[1],dtype=torch.double)

x_batch_i = torch.cartesian_prod(x1, x2).unsqueeze(1)

X1 = x_batch_i[:, 0, 0].view(x_num[0], x_num[1])
X2 = x_batch_i[:, 0, 1].view(x_num[0], x_num[1])

x_batch = torch.cartesian_prod(x1, x2).unsqueeze(1).repeat(t_num, 1, 1)

value_numerical = solver.value_function(t_batch,x_batch)


In [55]:
x_batch.squeeze(1).shape

torch.Size([30000, 2])

In [56]:
x_samples.shape

torch.Size([1000, 2])

In [60]:
t_batch.unsqueeze(1).shape

torch.Size([30000, 1])

In [ ]:
A = model(t_batch, x_batch.squeeze(1))

In [9]:
A = model(t_batch.unsqueeze(1), x_batch.squeeze(1))
B = torch.abs(A - value_numerical)
B

In [37]:
t_samples.shape

x_samples.shape

torch.Size([1000, 2])

In [25]:
# Assume we're evaluating at specific t and across a grid in x
t_eval = torch.tensor([[0.5]])  # Example: Evaluate at t=0.5
x_eval_grid = ...  # Create a grid of x values within the domain

with torch.no_grad():
    u_pred = model(t_eval, x_eval_grid)
    # Visualization or comparison code here


TypeError: expected Tensor as element 1 in argument 0, but got ellipsis